In [1]:
import pandas as pd 
import sqlalchemy
from binance.client import Client
import matplotlib.pyplot as plt

%matplotlib inline

In [2]:
api_key, api_secret = "", ""

In [3]:
client = Client(api_key, api_secret)

In [4]:
engine = sqlalchemy.create_engine('sqlite:///BTCUSDTstream.db')


In [5]:
df = pd.read_sql('BTCUSDT', engine)
df.head()

,symbol,Time,Price
0,BTCUSDT,2023-07-29 10:17:58.627,29270.00
1,BTCUSDT,2023-07-29 10:17:58.655,29269.99
2,BTCUSDT,2023-07-29 10:17:58.655,29269.99
3,BTCUSDT,2023-07-29 10:17:58.655,29269.99
4,BTCUSDT,2023-07-29 10:17:59.979,29270.00


In [6]:
# Trendfollowing - X/Twitter
# if the crypto was rising by x % -> Buy
# exit when profit is above 0.15% or loss is crossing -0.15%
# Lookback defines the number of timesteps to go back to.
# 1 timestep = 1 second

In [10]:
x = 2
y =[x]
y

[2]

In [8]:
def strategy(entry, lookback, qty, open_position=False):
    while True:
        df = pd.read_sql('BTCUSDT', engine)
        loockbackperiod = df.iloc[-lookback:]
        cumret = (loockbackperiod.Price.pct_change() +1).cumprod() -1
        if not open_position:
            if cumret[cumret.last_valid_index()] > entry:
                order = client.create_order(symbol='BTCUSDT',
                                            side='BUY',
                                            type='MARKET',
                                            quantity=qty)
                print(order)
                open_position = True
                break
    
    if open_position:
        while True:
            df = pd.read_sql('BTCUSDT', engine)
            sincebuy = df.loc[df.Time > pd.to_datetime(order['transactionTime'],
                                                       unit='ms')]
            
            if len(sincebuy) > 1:
                sincebuyret = (sincebuy.Price.pct_change() +1).cumprod() -1
                last_entry = sincebuyret[sincebuyret.last_valid_index()]
                if last_entry > 0.0015 or last_entry < -0.0015:
                    order = client.create_order(symbol='BTCUSDT',
                                            side='SELL',
                                            type='MARKET',
                                            quantity=qty)
                    print(order)
                    break

In [ ]:
strategy(0.001, 60, 0.001)

### What is cumulative product? Give a simple example

#### Source - ChatGPT
Cumulative product is a mathematical operation that involves multiplying a sequence of numbers together one by one, starting from the beginning of the sequence and proceeding to the end. The result of each multiplication is then used as one of the factors in the next multiplication, creating a cumulative effect.

For example, let's consider the sequence of numbers: 2, 3, 4, and 5.

The cumulative product of this sequence would be calculated as follows:

Start with the first number: 2 (Cumulative product so far: 2).
Multiply the second number with the cumulative product so far: 2 * 3 = 6 (Cumulative product so far: 6).
Multiply the third number with the cumulative product so far: 6 * 4 = 24 (Cumulative product so far: 24).
Multiply the fourth number with the cumulative product so far: 24 * 5 = 120.
The final result, 120, is the cumulative product of the sequence {2, 3, 4, 5}.

### How is it used in trading

In trading and finance, cumulative product is used in the context of calculating the cumulative returns of an investment over a certain period. It helps investors and traders understand the overall growth or decline of an investment over time.

#### Here's how it works:

Daily Returns: In trading, investors often track the daily returns of an asset or investment. Daily returns are calculated as the percentage change in the asset's value from one trading day to the next. For example, if the price of a stock increased from $100 to $110, the daily return would be (110 - 100) / 100 = 0.10 or 10%.

> Cumulative Product of Returns: To calculate the cumulative return over a period, the daily returns are used to compute the cumulative product. The formula is as follows:

> ```Cumulative Return = (1 + Return_1) * (1 + Return_2) * ... * (1 + Return_n)```

> Where Return_1, Return_2, ..., Return_n are the daily returns for each trading day within the specified period.

Example: Let's say you have daily returns for a stock over five consecutive trading days as follows: 0.02, 0.03, -0.01, 0.015, and 0.02. To calculate the cumulative return over these five days, you would apply the formula:

> ```Cumulative Return = (1 + 0.02) * (1 + 0.03) * (1 - 0.01) * (1 + 0.015) * (1 + 0.02) ≈ 1.0667```

The cumulative return in this example is approximately 1.0667 or 6.67%. This means that the investment has grown by about 6.67% over the five-day period.

Investors and traders use cumulative returns to assess the performance of their investments over various timeframes. It helps them understand the overall trend and performance of their portfolio, which can be essential in making informed decisions about their trading strategies. However, it's important to note that cumulative returns don't provide information about the volatility or risk of the investment, and other metrics like standard deviation or Sharpe ratio should be considered to get a more comprehensive view of the investment's performance.